In [1]:
#https://scrapfly.io/blog/web-scraping-with-selenium-and-python/
# https://stackoverflow.com/questions/53475578/python-selenium-multiprocessing

TODO
- явно есть ошибки не корретного парсинга, можно найти кейсы по дешевым билетам
- подумать может сделать без селениума
- на сервер запустить скрипт

In [2]:
import os
import json
import asyncio
from datetime import datetime, timedelta
from concurrent.futures.thread import ThreadPoolExecutor

import nest_asyncio
from utils.aviasales_parser import parse_page, list_dates, get_ticket, conver_all_json

nest_asyncio.apply()

In [3]:
from itertools import product
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")



In [4]:
def scrape(origin, destination, datetime_object, *, loop):
    loop.run_in_executor(executor, get_ticket, origin, destination, datetime_object)

In [5]:
executor = ThreadPoolExecutor(3)
loop = asyncio.get_event_loop()


destinations =  ['LED', 'MOW', 'KZN'] #['BUD', 'TSR', 'OSI', 'PEV', 'SOB', 'DEB', 'ZAG', 'BEG', 'TGD'] #['LED', 'MOW', 'KZN']
origins = ['BEG', 'TGD', 'TIV', 'TIA' ] #['HEL', 'TLL']# ['BEG', 'TGD', 'TIA', 'TIV', 'BUD', 'TSR', 'OSI', 'PEV', 'SOB', 'DEB', 'ZAG']
range_date = ['2024-04-20', '2024-09-30']
number_ticket = 1

date_start, date_end = range_date[0], range_date[1]
datetime_start = datetime.strptime(date_start, '%Y-%m-%d')
datetime_end = datetime.strptime(date_end, '%Y-%m-%d')
range_dates = list_dates(datetime_start, datetime_end)

parameters = list(product(origins, destinations, list_dates(datetime_start, datetime_end)))

# билеты туда
for origin, destination, datetime_object in tqdm(parameters, total=len(parameters), desc='parse ticket'):
    scrape(origin, destination, datetime_object, loop=loop)


origins, destinations = destinations, origins
parameters = list(product(origins, destinations, list_dates(datetime_start, datetime_end)))

for origin, destination, datetime_object in tqdm(parameters, total=len(parameters), desc='parse ticket'):
    scrape(origin, destination, datetime_object, loop=loop)

parse ticket:   0%|          | 0/1968 [00:00<?, ?it/s]

parse ticket:   0%|          | 0/1968 [00:00<?, ?it/s]

In [6]:
conver_all_json()